In [25]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.dates as mdates
from typing import List, Dict, Optional, Tuple

# 全局配置
pd.set_option('display.max_columns', None)
plt.rcParams["font.family"] = ["SimHei", "Microsoft YaHei", "Arial Unicode MS"]
plt.rcParams['axes.unicode_minus'] = False

# 核心字段（新增买一/卖一挂单字段，用于精准计算买卖盘比）
CORE_FIELDS = [
    # 基础信息
    "date", "stock_code", "stock_name", "open", "close", "pre_close",
    # 竞价数据
    "auc_volume", "auc_volume_ratio_vs_5d_avg",
    # 量能数据
    "volume", "volume_ratio_vs_5d_avg", "money_ratio_vs_5d_avg",
    # 盘口数据（新增买一/卖一挂单量，修正买卖盘比计算）
    "buy_total", "sell_total", "b1_v", "a1_v", "order_book_volume_ratio",
    # 波动数据
    "volatility", "amplitude",
    # 行业分类
    "sw_l1_industry_name"
]

class OptimizedIntradayStrategy:
    def __init__(self, data_path: str, 
                 output_dir: str = "optimized_selection_results",
                 eval_dir: str = "optimized_strategy_evaluation"):
        """初始化优化版超短选股策略（修正指标+行业逻辑）"""
        self.data_path = data_path
        self.output_dir = output_dir
        self.eval_dir = eval_dir
        
        # 创建输出目录
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.eval_dir, exist_ok=True)
        
        # 数据存储
        self.raw_data = None
        self.processed_data = None
        self.trading_dates = []
        self.trading_dates_map = {}  # 日期→索引映射，提升效率
        
        # 策略结果
        self.daily_selections = {}
        self.trade_records = None
        self.performance = None

    def load_data(self) -> bool:
        """加载数据（新增买一/卖一字段检查）"""
        try:
            print("加载优化版超短选股数据...")
            # 支持CSV/Parquet格式
            if self.data_path.endswith('.csv'):
                self.raw_data = pd.read_csv(self.data_path)
            elif self.data_path.endswith('.parquet'):
                self.raw_data = pd.read_parquet(self.data_path)
            else:
                raise ValueError("仅支持CSV或Parquet格式数据")
            
            # 检查核心字段（含买一/卖一挂单量）
            missing_fields = [f for f in CORE_FIELDS if f not in self.raw_data.columns]
            if missing_fields:
                raise ValueError(f"缺失核心字段: {missing_fields}")
            
            # 数据预处理
            df = self.raw_data[CORE_FIELDS].copy()
            # 日期格式统一
            df['date'] = pd.to_datetime(df['date'], errors='coerce').dropna()
            # 数值字段转换
            numeric_fields = [f for f in CORE_FIELDS if f not in 
                             ["date", "stock_code", "stock_name", "sw_l1_industry_name"]]
            for field in numeric_fields:
                df[field] = pd.to_numeric(df[field], errors='coerce')
            
            # 过滤无效数据（确保可交易）
            initial_count = len(df)
            df = df[
                (df['auc_volume'] > 0) &  # 有竞价数据
                (df['volume'] > 0) &       # 有当日成交
                (df['open'] > 0) &        # 有效开盘价（T+1日可买入）
                (df['close'] > 0) &       # 有效收盘价
                (df['b1_v'] > 0) &         # 有买一挂单
                (df['a1_v'] > 0)          # 有卖一挂单
            ].dropna(subset=numeric_fields + ["date"])
            
            # 按股票+日期排序
            df = df.sort_values(['stock_code', 'date']).reset_index(drop=True)
            self.processed_data = df
            
            # 生成交易日映射
            self.trading_dates = sorted(df['date'].unique())
            self.trading_dates_map = {date: idx for idx, date in enumerate(self.trading_dates)}
            
            print(f"数据预处理完成: 保留 {len(self.processed_data)} 条记录，{len(self.trading_dates)} 个交易日")
            return True
        except Exception as e:
            print(f"数据加载失败: {str(e)}")
            return False

    def calculate_daily_percentiles(self, date_data: pd.DataFrame) -> pd.DataFrame:
        """计算每日个股百分位（核心优化：修正买卖盘比、明确高开幅度价值）"""
        df = date_data.copy()
        
        # 1. 计算核心衍生指标（修正+新增）
        # （1）高开幅度：用于T+1日买入安全性判断（0.3%-5%最优）
        df['open_pct'] = (df['open'] - df['pre_close']) / df['pre_close'] * 100
        # （2）盘口量比：买一至买五总量 ÷ 卖一至卖五总量（反映整体多空预期）
        df['order_book_volume_ratio'] = df['buy_total'] / df['sell_total'].clip(lower=1)
        # （3）买卖盘比：买一挂单量 ÷ 卖一挂单量（反映瞬时多空，修正后公式）
        df['buy_sell_ratio'] = df['b1_v'] / df['a1_v'].clip(lower=1)
        
        # 2. 百分位评分维度（7个指标，权重100分，贴合超短需求）
        percentile_fields = [
            # 资金关注度（40分）：竞价+成交量，反映资金是否关注
            ("auc_volume_ratio_vs_5d_avg", 1, 20),  # 竞价量5日比：越高越好
            ("volume_ratio_vs_5d_avg", 1, 20),     # 成交量5日比：越高越好
            # 多空力量（30分）：总量+瞬时，双重验证买盘强度
            ("order_book_volume_ratio", 1, 15),    # 盘口量比：总量买盘占优
            ("buy_sell_ratio", 1, 15),             # 买卖盘比：瞬时买盘占优
            # 风险控制（30分）：高开+波动，避免踩坑
            ("open_pct", 1, 10),                   # 高开幅度：0.3%-5%最优
            ("volatility", -1, 10),                # 20日波动率：越低越稳
            ("amplitude", -1, 10)                  # 当日振幅：越低越安全
        ]
        
        # 3. 计算百分位及加权分
        total_score = 0
        for field, direction, weight in percentile_fields:
            # 特殊处理：高开幅度（0.3%-5%最优，超出部分扣分）
            if field == "open_pct":
                # 先将高开幅度clip到0.3-5%，再算百分位
                clipped_field = df[field].clip(0.3, 5)
                df[f'{field}_pctile'] = clipped_field.rank(pct=True) * 100
            else:
                # 普通字段：正向（越高分高）/反向（越低分高）
                rank_pct = df[field].rank(pct=True)
                df[f'{field}_pctile'] = rank_pct * 100 if direction == 1 else (1 - rank_pct) * 100
            
            # 加权分计算
            df[f'{field}_score'] = df[f'{field}_pctile'] * (weight / 100)
            total_score += df[f'{field}_score']
        
        # 4. 综合评分（保留1位小数）
        df['composite_score'] = total_score.round(1)
        
        return df

    def select_stocks_by_percentile(self, date: datetime, 
                                   top_quality_pct: float = 0.2,  # 前20%优质股
                                   top_industry_n: int = 3,      # 选前3个强势行业
                                   top_stock_per_industry: int = 2,  # 每行业选前2只
                                   max_total: int = 6) -> Optional[pd.DataFrame]:
        """优化版选股逻辑：先选强势行业，再选行业内个股（贴合A股轮动）"""
        # 1. 获取当日有效个股数据
        date_data = self.processed_data[self.processed_data['date'] == date].copy()
        if len(date_data) < 20:  # 个股数量不足，避免样本偏差
            print(f"{date.strftime('%Y-%m-%d')} 当日个股不足20只，跳过选股")
            return None
        
        # 2. 计算个股百分位及综合评分
        date_data = self.calculate_daily_percentiles(date_data)
        
        # 3. 步骤1：筛选前20%优质股（综合评分前20%）
        quality_threshold = date_data['composite_score'].quantile(1 - top_quality_pct)  # 80分位数
        high_quality_stocks = date_data[date_data['composite_score'] >= quality_threshold].copy()
        if len(high_quality_stocks) == 0:
            print(f"{date.strftime('%Y-%m-%d')} 无符合条件的优质股，跳过")
            return None
        
        # 4. 步骤2：找强势行业（优质股数量最多的前N个行业）
        # 统计每个行业的优质股数量
        industry_quality_count = high_quality_stocks.groupby('sw_l1_industry_name').agg({
            'stock_code': 'count',  # 优质股数量
            'composite_score': 'mean'  # 行业平均评分（避免数量少但评分低的行业）
        }).reset_index()
        industry_quality_count.columns = ['industry_name', 'quality_count', 'avg_score']
        
        # 筛选：优质股数量≥2，且按数量降序取前N个行业
        valid_industries = industry_quality_count[industry_quality_count['quality_count'] >= 2]
        if len(valid_industries) < top_industry_n:
            # 若有效行业不足，按平均评分补充
            top_industries = industry_quality_count.nlargest(top_industry_n, 'avg_score')['industry_name'].tolist()
        else:
            top_industries = valid_industries.nlargest(top_industry_n, 'quality_count')['industry_name'].tolist()
        
        print(f"{date.strftime('%Y-%m-%d')} 强势行业TOP{len(top_industries)}：{top_industries}")
        
        # 5. 步骤3：每个强势行业选前1-2只优质股
        selected = []
        for industry in top_industries:
            industry_stocks = high_quality_stocks[high_quality_stocks['sw_l1_industry_name'] == industry]
            # 行业内按综合评分降序，取前N只
            top_stocks = industry_stocks.nlargest(min(top_stock_per_industry, len(industry_stocks)), 'composite_score')
            selected.append(top_stocks)
        
        # 6. 合并结果，控制总数量（最多6只，避免过度分散）
        final_selected = pd.concat(selected, ignore_index=True)
        final_selected = final_selected.nlargest(min(max_total, len(final_selected)), 'composite_score')
        
        # 7. 格式化输出字段（含T+1日买入参考信息）
        output_cols = [
            'date', 'stock_code', 'stock_name', 'sw_l1_industry_name',
            'open', 'open_pct', 'composite_score',
            'auc_volume_ratio_vs_5d_avg', 'volume_ratio_vs_5d_avg',
            'order_book_volume_ratio', 'buy_sell_ratio',
            'volatility', 'amplitude'
        ]
        result = final_selected[output_cols].copy()
        
        # 重命名列名（突出T+1日价值）
        result.columns = [
            '选股日期', '股票代码', '股票名称', '所属强势行业',
            'T日开盘价', 'T日高开幅度(%)', '综合评分',
            'T日竞价量5日比', 'T日成交量5日比',
            'T日盘口量比(总量)', 'T日买卖盘比(瞬时)',
            'T日20日波动率', 'T日当日振幅'
        ]
        
        # 数值格式化
        float_cols = ['T日高开幅度(%)', 'T日竞价量5日比', 'T日成交量5日比', 
                     'T日盘口量比(总量)', 'T日买卖盘比(瞬时)', 'T日20日波动率', 'T日当日振幅']
        for col in float_cols:
            result[col] = result[col].round(3)
        
        print(f"{date.strftime('%Y-%m-%d')} 选股完成：共选出 {len(result)} 只个股（强势行业内TOP{top_stock_per_industry}）")
        return result

    def run_daily_selection(self, start_date: Optional[str] = None, end_date: Optional[str] = None) -> bool:
        """执行每日选股（排除最后1个交易日，确保T+1日可买入）"""
        target_dates = self.trading_dates
        if start_date:
            start = pd.to_datetime(start_date)
            target_dates = [d for d in target_dates if d >= start]
        if end_date:
            end = pd.to_datetime(end_date)
            target_dates = [d for d in target_dates if d <= end]
        
        # 排除最后1个交易日（避免T+1日无买入日期）
        if len(target_dates) > 1:
            target_dates = target_dates[:-1]
        else:
            print("无足够交易日（需至少2个：T日选股+T+1日买入）")
            return False
        
        print(f"开始每日选股：{len(target_dates)} 个交易日（聚焦强势行业）")
        
        all_results = []
        for date in tqdm(target_dates, desc="选股进度"):
            try:
                daily_result = self.select_stocks_by_percentile(date)
                if daily_result is not None and not daily_result.empty:
                    self.daily_selections[date] = daily_result
                    all_results.append(daily_result)
                    
                    # 保存每日结果
                    date_str = date.strftime("%Y%m%d")
                    daily_result.to_csv(
                        os.path.join(self.output_dir, f"selected_{date_str}.csv"),
                        index=False, encoding="utf-8-sig"
                    )
            except Exception as e:
                print(f"{date.strftime('%Y-%m-%d')} 选股出错: {str(e)}")
                continue
        
        if all_results:
            # 保存合并结果
            all_selected = pd.concat(all_results, ignore_index=True)
            all_selected.to_csv(
                os.path.join(self.output_dir, "all_selected_stocks.csv"),
                index=False, encoding="utf-8-sig"
            )
            print(f"选股完成：累计选出 {len(all_selected)} 只个股，覆盖 {len(all_results)} 个交易日")
            return True
        else:
            print("未选出任何个股（所有日期均无足够优质股）")
            return False

    def evaluate_strategy(self, hold_days: int = 1) -> bool:
        """评测策略收益（T日选股→T+1日买入→T+2日卖出）"""
        if not self.daily_selections:
            print("无选股结果，无法评测")
            return False
        if hold_days < 1:
            print("持有天数需≥1（超短策略至少持有1天）")
            return False
        
        print(f"开始策略评测：T日选股→T+1日买入→持有{hold_days}天→T+1+{hold_days}日卖出")
        records = []
        
        for select_date, selections in tqdm(self.daily_selections.items(), desc="计算收益"):
            # 1. 确定买卖日期（T+1买入，T+1+hold_days卖出）
            if select_date not in self.trading_dates_map:
                continue
            select_idx = self.trading_dates_map[select_date]
            
            # 买入日期（T+1）
            buy_idx = select_idx + 1
            if buy_idx >= len(self.trading_dates):
                continue
            buy_date = self.trading_dates[buy_idx]
            
            # 卖出日期（T+1+hold_days）
            sell_idx = buy_idx + hold_days
            if sell_idx >= len(self.trading_dates):
                continue
            sell_date = self.trading_dates[sell_idx]
            
            # 2. 模拟每只个股的交易
            for _, stock in selections.iterrows():
                stock_code = stock['股票代码']
                
                # 获取买入价格（T+1日开盘价）
                buy_data = self.processed_data[
                    (self.processed_data['stock_code'] == stock_code) &
                    (self.processed_data['date'] == buy_date)
                ]
                if buy_data.empty:
                    continue
                buy_price = buy_data['open'].values[0]
                
                # 获取卖出价格（T+1+hold_days日收盘价）
                sell_data = self.processed_data[
                    (self.processed_data['stock_code'] == stock_code) &
                    (self.processed_data['date'] == sell_date)
                ]
                if sell_data.empty:
                    continue
                sell_price = sell_data['close'].values[0]
                
                # 计算收益
                profit_pct = (sell_price - buy_price) / buy_price * 100
                profit_ratio = round(profit_pct, 2)
                
                # 记录交易（含强势行业标签）
                records.append({
                    '选股日期': select_date,
                    '买入日期': buy_date,
                    '卖出日期': sell_date,
                    '股票代码': stock_code,
                    '股票名称': stock['股票名称'],
                    '所属强势行业': stock['所属强势行业'],
                    '选股时综合评分': stock['综合评分'],
                    'T日高开幅度(%)': stock['T日高开幅度(%)'],
                    '买入价格(元)': round(buy_price, 2),
                    '卖出价格(元)': round(sell_price, 2),
                    '持有天数': hold_days,
                    '收益率(%)': profit_ratio
                })
        
        if not records:
            print("无有效交易记录用于评测")
            return False
        
        # 整理交易记录与指标
        self.trade_records = pd.DataFrame(records)
        total_trades = len(self.trade_records)
        profitable = (self.trade_records['收益率(%)'] > 0).sum()
        win_rate = round(profitable / total_trades * 100, 2) if total_trades > 0 else 0
        
        # 按强势行业统计表现
        industry_perf = self.trade_records.groupby('所属强势行业').agg({
            '收益率(%)': ['count', 'mean', lambda x: (x>0).mean()*100],
            '选股时综合评分': 'mean'
        }).round(2)
        industry_perf.columns = ['交易次数', '平均收益率(%)', '胜率(%)', '平均综合评分']
        
        # 计算累计收益（按买入日期排序，复利计算）
        self.trade_records = self.trade_records.sort_values('买入日期').reset_index(drop=True)
        self.trade_records['累计收益(%)'] = ((1 + self.trade_records['收益率(%)']/100).cumprod() - 1) * 100
        
        # 整体表现统计
        self.performance = {
            '总交易次数': total_trades,
            '盈利次数': profitable,
            '胜率(%)': win_rate,
            '平均收益率(%)': round(self.trade_records['收益率(%)'].mean(), 2),
            '累计收益率(%)': round(self.trade_records['累计收益(%)'].iloc[-1], 2),
            '最大盈利(%)': round(self.trade_records['收益率(%)'].max(), 2),
            '最大亏损(%)': round(self.trade_records['收益率(%)'].min(), 2),
            '收益标准差(%)': round(self.trade_records['收益率(%)'].std(), 2),
            '回测期间': f"{self.trade_records['买入日期'].min().strftime('%Y-%m-%d')} 至 {self.trade_records['买入日期'].max().strftime('%Y-%m-%d')}"
        }
        
        # 保存评测结果
        self.trade_records.to_csv(
            os.path.join(self.eval_dir, "trade_records.csv"),
            index=False, encoding="utf-8-sig"
        )
        
        with pd.ExcelWriter(os.path.join(self.eval_dir, "strategy_performance.xlsx")) as writer:
            pd.DataFrame([self.performance]).to_excel(writer, sheet_name="整体统计", index=False)
            industry_perf.to_excel(writer, sheet_name="强势行业表现", index=True)
            self.trade_records.sort_values('收益率(%)', ascending=False).head(10).to_excel(writer, sheet_name="最佳个股", index=False)
        
        # 打印结果
        print("\n===== 优化版超短策略评测结果 =====")
        print(f"交易规则: T日选强势行业个股 → T+1日开盘买入 → 持有{hold_days}天卖出")
        print(f"回测期间: {self.performance['回测期间']}")
        print("\n----- 核心指标 -----")
        for key in ['总交易次数', '胜率(%)', '平均收益率(%)', '累计收益率(%)', '最大盈利(%)', '最大亏损(%)']:
            print(f"{key}: {self.performance[key]}")
        
        print("\n----- 强势行业表现TOP3 -----")
        print(industry_perf.sort_values('平均收益率(%)', ascending=False).head(3).to_string())
        
        return True

    def plot_performance(self) -> None:
        """可视化策略表现（突出强势行业收益）"""
        if self.trade_records is None or self.performance is None:
            print("无评测数据，无法绘图")
            return
        
        plt.figure(figsize=(15, 12))
        
        # 子图1：累计收益率走势
        plt.subplot(3, 1, 1)
        plt.plot(
            self.trade_records['买入日期'],
            self.trade_records['累计收益(%)'],
            'g-', linewidth=2.5,
            label=f'累计收益率: {self.performance["累计收益率(%)"]}%'
        )
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.6)
        plt.fill_between(
            self.trade_records['买入日期'],
            self.trade_records['累计收益(%)'],
            0,
            where=(self.trade_records['累计收益(%)'] >= 0),
            color='lightgreen', alpha=0.3
        )
        plt.title('优化版超短策略累计收益率走势（按买入日期）', fontsize=14)
        plt.ylabel('累计收益率(%)', fontsize=12)
        plt.grid(alpha=0.3)
        plt.legend()
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2))
        plt.gcf().autofmt_xdate()
        
        # 子图2：强势行业平均收益率对比
        plt.subplot(3, 1, 2)
        industry_avg = self.trade_records.groupby('所属强势行业')['收益率(%)'].mean().reset_index()
        industry_avg = industry_avg.sort_values('收益率(%)', ascending=False).head(5)  # 取前5个行业
        
        bars = plt.bar(
            industry_avg['所属强势行业'],
            industry_avg['收益率(%)'],
            color=['darkgreen', 'green', 'lightgreen', 'gold', 'orange']
        )
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.6)
        plt.title('强势行业平均收益率TOP5', fontsize=14)
        plt.ylabel('平均收益率(%)', fontsize=12)
        plt.grid(alpha=0.3, axis='y')
        
        # 添加数值标签
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + (0.05 if height>0 else -0.15),
                     f'{height:.2f}%', ha='center', va='bottom' if height>0 else 'top')
        
        # 子图3：高开幅度与收益率关系（验证T+1日价值）
        plt.subplot(3, 1, 3)
        # 按高开幅度分组
        self.trade_records['高开幅度区间'] = pd.cut(
            self.trade_records['T日高开幅度(%)'],
            bins=[0, 0.3, 1, 3, 5, 10],
            labels=['0-0.3%', '0.3-1%', '1-3%', '3-5%', '5-10%']
        )
        open_pct_perf = self.trade_records.groupby('高开幅度区间')['收益率(%)'].agg(['mean', 'count']).reset_index()
        
        bars = plt.bar(
            open_pct_perf['高开幅度区间'],
            open_pct_perf['mean'],
            alpha=0.8,
            color='skyblue'
        )
        plt.axhline(y=0, color='r', linestyle='--', alpha=0.6)
        plt.title('T日高开幅度区间与T+1日交易收益率关系', fontsize=14)
        plt.xlabel('T日高开幅度区间', fontsize=12)
        plt.ylabel('平均收益率(%)', fontsize=12)
        plt.grid(alpha=0.3, axis='y')
        
        # 添加数值标签（含交易次数）
        for i, (_, row) in enumerate(open_pct_perf.iterrows()):
            plt.text(i, row['mean'] + (0.05 if row['mean']>0 else -0.15),
                     f'{row["mean"]:.2f}%\n(N={int(row["count"])})',
                     ha='center', va='bottom' if row['mean']>0 else 'top')
        
        plt.tight_layout(pad=3.0)
        plt.savefig(
            os.path.join(self.eval_dir, 'strategy_performance.png'),
            dpi=300,
            bbox_inches='tight'
        )
        plt.close()
        
        print(f"\n策略表现图表已保存至: {self.eval_dir}/strategy_performance.png")

# 主函数入口（实盘逻辑）
if __name__ == "__main__":
    # 配置参数（替换为你的实际路径）
    DATA_PATH = r"D:\workspace\xiaoyao\data\factortable.parquet"  # 数据文件路径
    START_DATE = "2025-01-01"  # 选股开始日期
    END_DATE = "2025-09-20"    # 选股结束日期（提前2天，确保卖出）
    HOLD_DAYS = 1              # 超短持有天数（1天）
    
    # 启动策略
    print("=== 启动优化版超短选股策略（贴合A股轮动） ===")
    strategy = OptimizedIntradayStrategy(DATA_PATH)
    
    # 执行流程：加载数据→每日选股→收益评测→可视化
    if strategy.load_data():
        if strategy.run_daily_selection(start_date=START_DATE, end_date=END_DATE):
            if strategy.evaluate_strategy(hold_days=HOLD_DAYS):
                strategy.plot_performance()
    
    print(f"\n所有流程完成!")
    print(f"选股结果目录: {strategy.output_dir}（每日文件含强势行业标签）")
    print(f"评测结果目录: {strategy.eval_dir}（含行业收益对比）")

=== 启动优化版超短选股策略（贴合A股轮动） ===
加载优化版超短选股数据...
数据预处理完成: 保留 920710 条记录，182 个交易日
开始每日选股：174 个交易日（聚焦强势行业）


选股进度:   1%|          | 1/174 [00:00<00:31,  5.48it/s]

2025-01-03 强势行业TOP3：['医药生物I', '化工I', '机械设备I']
2025-01-03 选股完成：共选出 6 只个股（强势行业内TOP2）


选股进度:   2%|▏         | 3/174 [00:00<00:19,  8.72it/s]

2025-01-06 强势行业TOP3：['医药生物I', '机械设备I', '化工I']
2025-01-06 选股完成：共选出 6 只个股（强势行业内TOP2）
2025-01-07 强势行业TOP3：['医药生物I', '电子I', '机械设备I']
2025-01-07 选股完成：共选出 6 只个股（强势行业内TOP2）
2025-01-08 强势行业TOP3：['机械设备I', '化工I', '医药生物I']
2025-01-08 选股完成：共选出 6 只个股（强势行业内TOP2）


...

2025-09-17 强势行业TOP3：['机械设备I', '医药生物I', '汽车I']
2025-09-17 选股完成：共选出 6 只个股（强势行业内TOP2）
2025-09-18 强势行业TOP3：['机械设备I', '电子I', '医药生物I']
2025-09-18 选股完成：共选出 6 只个股（强势行业内TOP2）
选股完成：累计选出 1044 只个股，覆盖 174 个交易日
开始策略评测：T日选股→T+1日买入→持有1天→T+1+1日卖出


计算收益: 100%|██████████| 174/174 [02:32<00:00,  1.14it/s]



===== 优化版超短策略评测结果 =====
交易规则: T日选强势行业个股 → T+1日开盘买入 → 持有1天卖出
回测期间: 2025-01-06 至 2025-09-19

----- 核心指标 -----
总交易次数: 1020
胜率(%): 50.88
平均收益率(%): 0.31
累计收益率(%): 1021.11
最大盈利(%): 32.02
最大亏损(%): -17.32

----- 强势行业表现TOP3 -----
        交易次数  平均收益率(%)  胜率(%)  平均综合评分
所属强势行业                               
计算机I      52      0.77  53.85   83.36
电气设备I     77      0.51  48.05   83.66
电子I      181      0.45  51.93   82.66


C:\Users\user\AppData\Local\Temp\ipykernel_13880\2867315193.py:476: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  open_pct_perf = self.trade_records.groupby('高开幅度区间')['收益率(%)'].agg(['mean', 'count']).reset_index()
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 'Arial Unicode MS' not found.
findfont: Font family 


策略表现图表已保存至: optimized_strategy_evaluation/strategy_performance.png

所有流程完成!
选股结果目录: optimized_selection_results（每日文件含强势行业标签）
评测结果目录: optimized_strategy_evaluation（含行业收益对比）


In [ ]:
import pandas as pd

# 读取交易记录文件
df = pd.read_csv(r'D:\workspace\xiaoyao\works\trytry\optimized_strategy_evaluation/trade_records.csv')

# 按选股日期分组，计算每日平均收益率
sydf = df.groupby('选股日期')['收益率(%)'].mean().reset_index()

# 重命名列名
sydf = sydf.rename(columns={'选股日期': 'date', '收益率(%)': 'return'})

# 查看结果
print(sydf.head())

         date    return
0  2025-01-03  0.260000
1  2025-01-06  2.933333
2  2025-01-07 -1.100000
3  2025-01-08 -1.916667
4  2025-01-09 -2.562500


In [23]:
# 用1+0.5*return后 得到收益增长
sydf['return'] = 1 + 0.5 * sydf['return']/100
sydf

,date,return
0,2025-01-03,1.001300
1,2025-01-06,1.014667
2,2025-01-07,0.994500
3,2025-01-08,0.990417
4,2025-01-09,0.987187
...,...,...
169,2025-09-12,0.998962
170,2025-09-15,0.997333
171,2025-09-16,0.994175
172,2025-09-17,0.994825


In [24]:
# 将return依次连乘，得到累计收益率
sydf['return'] = sydf['return'].cumprod()
sydf

,date,return
0,2025-01-03,1.001300
1,2025-01-06,1.015986
2,2025-01-07,1.010398
3,2025-01-08,1.000715
4,2025-01-09,0.987893
...,...,...
169,2025-09-12,1.309799
170,2025-09-15,1.306306
171,2025-09-16,1.298697
172,2025-09-17,1.291976
